# Klasifikasi teks metode naive bayes

In [11]:
import tkinter as tk
import tkinter.font as font
from tkinter import *
from tkinter import ttk
from time import strftime
from tkinter import ttk, filedialog, messagebox
import pandas as pd
import string 
import re #regex library
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import tkinter.scrolledtext as tkst
#from time import strftime
import swifter
from tkinter import messagebox as msg
from pandastable import Table
from tkintertable import TableCanvas
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import matplotlib
#matplotlib.use("TkAgg")
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.backend_bases import key_press_handler
from typing import Callable

root = Tk()
s = ttk.Style()
s.configure("lb.TLabel", foreground="aliceblue", background="aliceblue")
label_files = ttk.Label(root,text="No File Selected", style="lb.TLabel")
label_files.place(rely=0, relx=0) 

def File_dialog():
    """This function will open the file explorer"""
    global filename
    filename = filedialog.askopenfilename(initialdir='C:/Tutorial',
                                          title="Select A File",
                                          filetype=(("xlsx files", "*.xlsx"), ("all files", "*.*")))
    
    s.configure("sn.TLabel", foreground="DeepSkyBlue",background="aliceblue")
    file_frame = tk.LabelFrame(root, text="Selected File Preprocessing" ,bd =0
                          ,bg ='aliceblue', fg = 'DeepSkyBlue')
    file_frame.place(height=45, width=800, y=100, x=5)
    label_file = ttk.Label(file_frame, text=filename,style="sn.TLabel")
    label_file.place(y=0, x=0)
    label_files.configure(text=filename)
    menubar.entryconfig("Preprocessing", state="normal")
    """
    buttonplus = Button(root, text = 'Preprocessing', bd = '0', command = prins,
             bg = 'lightblue', fg = 'white')#, width=6, height=2)
    buttonplus.place(x=5, y=150)
    buttonshow = Button(root, text = 'Show Tabel', bd = '0', command = showtabel1,
             bg = 'lightblue', fg = 'white')#, width=6, height=2)
    buttonshow.place(x=135, y=150)
    buttonplus['font']=myFont
    buttonshow['font']=myFont
    """
    if filename =='':
        label_file.config(text="Tidak ada data")
    
    myFont = font.Font(family='Calibri', size=12, weight='bold')
    file_frame['font']=myFont
    label_file['font']=myFont
    
def showtabel1():
    excel_filename = r"{}".format(label_files['text'])
    review = pd.read_excel(excel_filename)
    newindow = Toplevel(root)
    newindow.title(filename)
    newindow.geometry('720x400')
    newindow.configure(bg="aliceblue")
    newindow.state("normal")
    f2 = Frame(newindow, height=200, width=300) 
    f2.pack(fill=BOTH,expand=1)
    table = Table(f2, dataframe=review,read_only=True)
    table.show()
    #excel_filename = r"{}".format(label_files['text'])
    #review = pd.read_excel(excel_filename)

def prins():
    excel_filename = r"{}".format(label_files['text'])
    global komen
    komen = pd.read_excel(excel_filename)
    global output
    output = pd.read_excel(excel_filename)
    global review
    review = pd.read_excel(excel_filename)
    if (len(review)== 0):
        print ('No records', 'No records')
    else:
        pass
    global labeld
    labeld = komen['label']
    global komentarasli
    komentarasli = komen['komentar']
    global komcasefold
    komcasefold = output['komentar'].str.lower()
    output['komentar']= output['komentar'].str.lower()
    review['komentar'] = review['komentar'].str.lower()
    
    def remove_komentar_special(text):
        # remove tab, new line, ans back slice
        text = str(text).replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
        # remove non ASCII (emoticon, chinese word, .etc)
        text = text.encode('ascii', 'replace').decode('ascii')
        # remove mention, link, hashtag
        text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
        # remove incomplete URL
        return text.replace("http://", " ").replace("https://", " ")
    review['komentar'] = review['komentar'].apply(remove_komentar_special)
    global komclean
    output['komentar']= output['komentar'].apply(remove_komentar_special)
    def remove_number(text):
        return  re.sub(r"\d+", "", text)
    
    output['komentar']= output['komentar'].apply(remove_number)
    review['komentar'] = review['komentar'].apply(remove_number)
    
    #remove punctuation
    def remove_punctuation(text):
        return text.translate(str.maketrans("","",string.punctuation))
    
    output['komentar']= output['komentar'].apply(remove_punctuation)
    review['komentar'] = review['komentar'].apply(remove_punctuation)

    #remove whitespace leading & trailing
    def remove_whitespace_LT(text):
        return text.strip()

    output['komentar']= output['komentar'].apply(remove_whitespace_LT)
    review['komentar'] = review['komentar'].apply(remove_whitespace_LT)

    #remove multiple whitespace into single whitespace
    def remove_whitespace_multiple(text):
        return re.sub('\s+',' ',text)
    
    output['komentar']= output['komentar'].apply(remove_whitespace_multiple)
    review['komentar'] = review['komentar'].apply(remove_whitespace_multiple)

    # remove single char
    def remove_singl_char(text):
        return re.sub(r"\b[a-zA-Z]\b", "", text)

    review['komentar'] = review['komentar'].apply(remove_singl_char)
    output['komentar']= output['komentar'].apply(remove_singl_char)
    
    # NLTK word Tokenize 
    def word_tokenize_wrapper(text):
        return word_tokenize(text)

    review['komentar_token'] = review['komentar'].apply(word_tokenize_wrapper)
    #print('Hasil Cleaning-Tokenizing : \n') 
    #print(review_data['komentar_token'].head(5))
    #print('\n\n\n')
    """
    # ------- Normalisasi -------

    normalizad_word = pd.read_excel("normalisasi.xlsx")

    normalizad_word_dict = {}

    for index, row in normalizad_word.iterrows():
        if row[0] not in normalizad_word_dict:
            normalizad_word_dict[row[0]] = row[1] 

    def normalized_term(document):
        return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

    review['komentar_normalisasi'] = review['komentar_token'].apply(normalized_term)
    """
    
    # ------- Stopword -------

    list_stopwords = stopwords.words('indonesia')

    # menambah additional stopword
    list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tuh', 'utk', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah','apa','pada','pakai','pengin','tepi'])

    # convert list to dictionary
    list_stopwords = set(list_stopwords)

    #remove stopword pada list token
    def stopwords_removal(words):
        return [word for word in words if word not in list_stopwords]

    review['komentar_token_stopwords'] = review['komentar_token'].apply(stopwords_removal) 
    
    # ------- Normalisasi -------

    normalizad_word = pd.read_excel("normalisasi.xlsx")

    normalizad_word_dict = {}

    for index, row in normalizad_word.iterrows():
        if row[0] not in normalizad_word_dict:
            normalizad_word_dict[row[0]] = row[1] 

    def normalized_term(document):
        return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

    review['komentar_normalisasi'] = review['komentar_token_stopwords'].apply(normalized_term)

    #review_data['komentar_normalisasi'].head()
    

    # ------- Stemming -------

    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # stemmed
    def stemmed_wrapper(term):
        return stemmer.stem(term)

    term_dict = {}

    for document in review['komentar_normalisasi']:
        for term in document:
            if term not in term_dict:
                term_dict[term] = ' '
            
    print(len(term_dict))
    print("------------------------")

    for term in term_dict:
        term_dict[term] = stemmed_wrapper(term)
        print(term,":" ,term_dict[term])
    
    print(term_dict)
    print("------------------------")


    # apply stemmed term to dataframe
    def get_stemmed_term(document):
        return [term_dict[term] for term in document]

    review['komentar_stemmed'] = review['komentar_normalisasi'].swifter.apply(get_stemmed_term)
    
    df_rows = review.to_numpy().tolist()
    print("gnissecorperp id lisahreb halet atad")
    #myFont = font.Font(family='Helvetica', size=12, weight='bold')
    #labelprep=Label(root, text="preprocessing berhasil", bg = "aliceblue").place(x=5, y=300)
    #labelprep['font']=myFont
    #labelpres=Frame(master = root, bg = 'aliceblue')
    #buttonshow.place(x=120, y=150)
    label_file = ttk.Label(root, text="Data berhasil diproses",style="sn.TLabel")
    label_file.place(x=5, y=150)
    
    myFont = font.Font(family='calibri', size=12, weight='bold')
    """
    buttonshow = Button(root, text = 'Show Tabel', bd = '0', command = showtabel2,
             bg = 'lightblue', fg = 'white')#, width=6, height=2)
    buttonshow.place(x=5, y=220)
   
    myFont = font.Font(family='Helvetica', size=12, weight='bold')
    """
    #buttonshow['font']=myFont
    label_file['font']=myFont
    #edit.entryconfig("Apply", state="normal")
    edit.entryconfig("Save Data", state="normal")
    edit.entryconfig("Hasil Preprocessing", state="normal")

def exportExcel ():
    
    export_file_path = filedialog.asksaveasfilename(defaultextension='.xlsx')
    review.to_excel (export_file_path, index = False, header=True)
    
def showtabel2():
    #print(review['komentar_stemmed'])

    dfkom = pd.DataFrame({
    'komentar': komentarasli,
    'label': labeld,
    'komentar_casefolding' : komcasefold,
    'komentar_cleaning': output['komentar'], 
    'komentar_tokenize': review['komentar_token'],
    'komentar_stopword': review['komentar_token_stopwords'],
    'komentar_normalisasi': review['komentar_normalisasi'],
    'komentar_stemmed': review['komentar_stemmed']
    })
    newindow2 = Toplevel(root)
    newindow2.title("Hasil preprocessing")
    #root.title('PandasTable Example')
    newindow2.geometry('720x400')
    newindow2.configure(bg="aliceblue")
    newindow2.state("normal")
    
    f2 = Frame(newindow2, height=200, width=300) 
    f2.pack(fill=BOTH,expand=1)
    table = Table(f2, dataframe=dfkom,read_only=True)
    table.show()

    
def helpp():
    msg.showinfo('Info',
                '-Jika ingin melakukan preprocessing harap menyiapkan file berformat xlsx atau excel\n-Untuk file excel yang di import/open harus memiliki kolom komentar\n-Setelah proses preprocessing data disave')

#global styl
styl = ttk.Style()
styl.configure("ide.TLabel", foreground="aliceblue", background="aliceblue")
global datalatih
datlatih = ttk.Label(root,text="No File Selected", styl="ide.TLabel")
datlatih.place(y=0, x=0)
def inputlatih():
    global filenamed
    filenamed = filedialog.askopenfilename(initialdir='C:/Tutorial',
                                          title="Select A File",
                                          filetype=(("xlsx files", "*.xlsx"), ("all files", "*.*")))
    styl.configure("ye.TLabel", foreground="DeepSkyBlue",background="aliceblue")
    datlatihg = tk.LabelFrame(root, text="Selected Data Latih" ,bd =0,bg ='aliceblue', fg = 'DeepSkyBlue')
    datlatihg.place(height=45, width=800, y=180, x=5)
    global datlatlabel
    datlatlabel = ttk.Label(datlatihg, text=filenamed,style="ye.TLabel")
    datlatlabel.place(y=0, x=0)
    datlatih.configure(text=filenamed)
    myFont = font.Font(family='Calibri', size=12, weight='bold')
    datlatihg['font']=myFont
    datlatlabel['font']=myFont
    if filenamed=="":
        datlatlabel.configure(text="Tidak ada data")
    #menubar.entryconfig("Klasifikasi Naive Bayes", state="normal")
    #sub_nevbayes.entryconfig(0, label = filenamed)
    
def converd():
    global conlat
    conlat = pd.DataFrame({
    'komentar': komentarasli,
    'label': labeld,
    'komentar_casefolding' : komcasefold,
    'komentar_cleaning': output['komentar'], 
    'komentar_tokenize': review['komentar_token'],
    'komentar_stopword': review['komentar_token_stopwords'],
    'komentar_mormalisasi': review['komentar_normalisasi'],
    'komentar_stemmed': review['komentar_stemmed']
    })
    styl.configure("ye.TLabel", foreground="DeepSkyBlue",background="aliceblue")
    datlatihg = tk.LabelFrame(root, text="Selected Data Latih" ,bd =0,bg ='aliceblue', fg = 'DeepSkyBlue')
    datlatihg.place(height=45, width=800, y=180, x=5)
    datlatlabel = ttk.Label(datlatihg, text="Data Latih Hasil Preprocessing",style="ye.TLabel")
    datlatlabel.place(y=0, x=0)
    myFont = font.Font(family='Calibri', size=12, weight='bold')
    datlatihg['font']=myFont
    datlatlabel['font']=myFont
    #menubar.entryconfig("Klasifikasi Naive Bayes", state="normal")
    sub_nevbayes.entryconfig(1, label = "Selected Data Latih")

def showcondat():
    df = conlat
    newindow = Toplevel(root)
    #newindow.title(datlatlabel)
    newindow.geometry('720x400')
    newindow.configure(bg="aliceblue")
    newindow.state("normal")
    f2 = Frame(newindow, height=200, width=300) 
    f2.pack(fill=BOTH,expand=1)
    table = Table(f2, dataframe=df,read_only=True)
    table.show()

def showdatlatih():
    global filedatlatih
    filedatlatih = r"{}".format(datlatih['text'])
    review = pd.read_excel(filedatlatih)
    #train_dokumen.drop(["no", "nama","komentar", "tanggal scraping (17-02-2021)", "komentar_token","komentar_token_stopwords", "komentar_normalisasi"], axis = 1, inplace = True) 
    #train_dokumen['label'] = train_dokumen['label'].map({'positif':1 , 'negatif':0})
    df = review[['komentar','label','komentar_token','komentar_token_stopwords','komentar_normalisasi','komentar_stemmed']]
    newindow = Toplevel(root)
    newindow.title(filenamed)
    newindow.geometry('720x400')
    newindow.configure(bg="aliceblue")
    newindow.state("normal")
    f2 = Frame(newindow, height=200, width=300) 
    f2.pack(fill=BOTH,expand=1)
    table = Table(f2, dataframe=df,read_only=True)
    table.show()
    

datuji = ttk.Label(root,text="No File Selected", styl="ide.TLabel")
datuji.place(y=0, x=0)
    
def inputuji():
    global fileuji
    fileuji = filedialog.askopenfilename(initialdir='C:/Tutorial',
                                          title="Select A File",
                                          filetype=(("xlsx files", "*.xlsx"), ("all files", "*.*")))
    
    styl.configure("ye.TLabel", foreground="DeepSkyBlue",background="aliceblue")
    datujis = tk.LabelFrame(root, text="Selected Data Uji" ,bd =0
                          ,bg ='aliceblue', fg = 'DeepSkyBlue')
    datujis.place(height=45, width=800, y=230, x=5)
    datujilabel = ttk.Label(datujis, text=fileuji,style="ye.TLabel")
    datujilabel.place(y=0, x=0)
    datuji.configure(text=fileuji)
    myFont = font.Font(family='Calibri', size=12, weight='bold')
    datujis['font']=myFont
    datujilabel['font']=myFont
    menubar.entryconfig("Klasifikasi Naive Bayes", state="normal")
    if fileuji=="":
        datujilabel.configure(text="Tidak ada data")
    #sub_nevbayes.entryconfig(3, label = fileuji)

def converduji():
    global conuj
    conuj = pd.DataFrame({
    'komentar': komentarasli,
    'label': labeld,
    'komentar_casefolding' : komcasefold,
    'komentar_cleaning': output['komentar'], 
    'komentar_tokenize': review['komentar_token'],
    'komentar_stopword': review['komentar_token_stopwords'],
    'komentar_mormalisasi': review['komentar_normalisasi'],
    'komentar_stemmed': review['komentar_stemmed']
    })
    styl.configure("ye.TLabel", foreground="DeepSkyBlue",background="aliceblue")
    datujis = tk.LabelFrame(root, text="Selected Data Uji" ,bd =0
                          ,bg ='aliceblue', fg = 'DeepSkyBlue')
    datujis.place(height=45, width=800, y=230, x=5)
    datujilabel = ttk.Label(datujis, text="Data Uji Hasil Preprocessing",style="ye.TLabel")
    datujilabel.place(y=0, x=0)
    myFont = font.Font(family='Calibri', size=12, weight='bold')
    datujis['font']=myFont
    datujilabel['font']=myFont
    menubar.entryconfig("Klasifikasi Naive Bayes", state="normal")
    sub_nevbayes.entryconfig(4, label = "Selected Data Uji")
    
def showconuji():
    df = conuj
    newindow = Toplevel(root)
    #newindow.title(fileuji)
    newindow.geometry('720x400')
    newindow.configure(bg="aliceblue")
    newindow.state("normal")
    f2 = Frame(newindow, height=200, width=300) 
    f2.pack(fill=BOTH,expand=1)
    table = Table(f2, dataframe=df,read_only=True)
    table.show()
        
def showdatuji():
    global filedata_uji
    filedata_uji = r"{}".format(datuji['text'])
    review = pd.read_excel(filedata_uji)
    df = review[['komentar','label','komentar_token','komentar_token_stopwords','komentar_normalisasi','komentar_stemmed']]
    newindow = Toplevel(root)
    newindow.title(fileuji)
    newindow.geometry('720x400')
    newindow.configure(bg="aliceblue")
    newindow.state("normal")
    f2 = Frame(newindow, height=200, width=300) 
    f2.pack(fill=BOTH,expand=1)
    table = Table(f2, dataframe=df,read_only=True)
    table.show()


    
class Model:
    def __init__(self, prob_c0, prob_c1, prob_w_c0, prob_w_c1):
        self.prob_c0 = prob_c0
        self.prob_c1 = prob_c1
        self.prob_w_c0 = prob_w_c0
        self.prob_w_c1 = prob_w_c1
def traindata(t):
    #x_train = []
    #y_train = []
    #if t.endswith('.xlsx'):
    #t2 = r"{}".format(t['text'])
    """
    if t == filedatlatih:
        train_dokumen = pd.read_excel(t)
    else:
        train_dokumen = conlat
    """
    train_dokumen = pd.read_excel(t)
    #train_dokumen.drop(["no", "nama","komentar", "tanggal scraping (17-02-2021)", "komentar_token","komentar_token_stopwords", "komentar_normalisasi"], axis = 1, inplace = True) 
    # mengganti label menjadi angka
    train_dokumen['label'] = train_dokumen['label'].map({'positif':1 , 'negatif':0})
    df = train_dokumen[['komentar_stemmed','label']]
    # hapus karakter
    df['komentar_stemmed'] = df['komentar_stemmed'].str.replace("[^a-zA-Z#]", " ")
    x_train_array = df.values
    x_train = x_train_array[:,0]
    y_train = x_train_array[:,1]
    # x_train merupakan list dokumen yang dijadikan training data
    # n merupakan banyak kata yang ingin ada di dalam vocabulary
        
    
    # menghitung frekuensi setiap kata
    from nltk.tokenize import word_tokenize
    
    titles = x_train
    freq = {}
    for t in titles:
        tokens = word_tokenize(t)
        for w in tokens:
            if w in freq:
                freq[w] += 1
            else:
                freq[w] = 1
    
    # mengurutkan kata berdasarkan frekuensinya
    sorted_freq = {key: values for key, values in sorted(freq.items(), key=lambda e: e[1], reverse=True)}
    
    # mengambil n kata dengan frekuensi terbesar untuk dijadikan vocabulary
    global vocab
    vocab = list(sorted_freq)[:1000]
    
    n_doc0 = list(y_train).count(0)
    n_doc1 = list(y_train).count(1)
    
    
    freq_0 = {}
    freq_1 = {}

    for i in range(len(x_train)):
        tokens = word_tokenize(x_train[i])

        # jika judul ke-i termasuk dalam kelas 0
        if y_train[i] == 0:
            for w in tokens:
                if w in freq_0:
                    freq_0[w] += 1
                else:
                    freq_0[w] = 1

        # jika judul ke-i termasuk dalam kelas 1
        else:
            for w in tokens:
                if w in freq_1:
                    freq_1[w] += 1
                else:
                    freq_1[w] = 1
    global fre
    fre = {key: values for key, values in sorted(freq_1.items(), key=lambda e: e[1], reverse=True)}

    global fre1
    fre1 = {key: values for key, values in sorted(freq_0.items(), key=lambda e: e[1], reverse=True)}
    
    k = pd.DataFrame({"positif":fre,"negatif":fre1})
    karray = k.values
    koskatneg = karray[:,1]
    koskatpos = karray[:,0]
    koskats = k.index
    global koskatable
    koskatable = pd.DataFrame({"Kosakata":koskats,"Negatif":koskatneg,"Positif":koskatpos})
    koskatable = koskatable.fillna(0)
    
    
    n_vocab0 = 0
    n_vocab1 = 0
    for w in vocab:
    # jika kata w muncul di kelas 0
        if w in freq_0:
            n_vocab0 += freq_0[w]
    # jika kata w muncul di kelas 1
        if w in freq_1:
            n_vocab1 += freq_1[w]    
    
    # menghitung banyak dokumen yang masuk ke setiap kelas di data training
    #n_doc0, n_doc1 = kelas_dokumen(y_train)
    
    # menghitung banyak dokumen pada data training
    global n_doc
    n_doc = len(x_train)
    
    # menghitung kemunculan setiap kata di setiap kelas pada data training
    #freq_0, freq_1 = frekuensikata(x_train, y_train)
        
    # menghitung banyak kata yang muncul di setiap kelas yang termasuk dalam vocabulary
    #n_vocab0, n_vocab1 = kelaskosakata(freq_0, freq_1, vocab)
    
    # menghitung estimasi
    # probabilitas prior kelas 0
    global prob_c0
    prob_c0 = n_doc0/n_doc

    # probabilitas prior kelas 1
    global prob_c1
    prob_c1 = n_doc1/n_doc
    
    # probabilitas kondisional
    global prob_w_c0
    global prob_w_c1
    prob_w_c0 = {}
    prob_w_c1 = {}

    for w in vocab:
        # jika kata w muncul di kelas 0
        if w in freq_0:
            prob_w_c0[w] = (freq_0[w]+1)/(n_vocab0+len(vocab))
        else:
            prob_w_c0[w] = 1/(n_vocab0+len(vocab))

        # jika kata w muncul di kelas 1
        if w in freq_1:
            prob_w_c1[w] = (freq_1[w]+1)/(n_vocab1+len(vocab))
        else:
            prob_w_c1[w] = 1/(n_vocab1+len(vocab))            
            
    return df,x_train,y_train,vocab,n_doc0,n_doc1,freq_0,freq_0,n_vocab0, n_vocab1, prob_c0, prob_c1, prob_w_c0, prob_w_c1
    
    
def training(x_train, y_train, nVocab, boolSmoothing):
    
    import pandas as pd
    
    # membuat vocab
    vocab
    
    # melakukan estimasi
    boolSmoothing = prob_c0, prob_c1, prob_w_c0, prob_w_c1
    model = Model(prob_c0, prob_c1, prob_w_c0, prob_w_c1)
    
    return model

def testdata(test, model):
    #if test.endswith('.xlsx'):
    #test2 = r"{}".format(test['text'])
    """
    if test==filedatuji:
        uji_dokumen = pd.read_excel(test)
    else:
        uji_dokumen = conuj
    """
    uji_dokumen = pd.read_excel(test)
    showdatest = pd.read_excel(test)
    #uji_dokumen.drop(["no", "nama","komentar", "tanggal scraping (17-02-2021)", "komentar_token","komentar_token_stopwords", "komentar_normalisasi"], axis = 1, inplace = True) 
    uji_dokumen['label'] = uji_dokumen['label'].map({'positif':1 , 'negatif':0})
    uji_dokumen = uji_dokumen[['komentar_stemmed','label']]
    uji_dokumen['komentar_stemmed'] = uji_dokumen['komentar_stemmed'].str.replace("[^a-zA-Z#]", " ")
    x_test_array = uji_dokumen.values
    x_test = x_test_array[:,0]
    y_test = x_test_array[:,1]
    global datest
    datest = showdatest['komentar']
    from nltk.tokenize import word_tokenize
    
    y = []

    for t in x_test:
        tokens = word_tokenize(t)
        # probabilitas prior
        p0 = model.prob_c0
        ask1 = model.prob_c1
        p1 = model.prob_c1
    

        # menghitung probabilitas title t masuk ke kelas 0 dan kelas 1
        for w in tokens:
            if w in model.prob_w_c0:
                p0 = p0 * model.prob_w_c0[w]

            if w in model.prob_w_c1:
                p1 = p1 * model.prob_w_c1[w]
        #print (p0,p1)
        # penentuan most likely class
        if p0 > p1:
            y.append(0)
        else:
            y.append(1)

    return y,y_test,x_test

def conf_matrix(y_real, y_predict, printMatrix):
    
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    tp0 = 0
    fp0 = 0
    tn0 = 0
    fn0 = 0
    tp1 = 0
    fp1 = 0
    tn1 = 0
    fn1 = 0

    for i in range(len(y_real)):
        # true positive
        if (y_real[i] == 1) & (y_predict[i] == 1):
            tn0 += 1
            tp1 += 1

        # false positive
        elif (y_real[i] == 0) & (y_predict[i] == 1):
            fn0 += 1
            fp1 += 1

        # true negative
        elif (y_real[i] == 0) & (y_predict[i] == 0):
            tp0 += 1
            tn1 += 1

        # false negative
        elif (y_real[i] == 1) & (y_predict[i] == 0):
            fp0 += 1
            fn1 += 1
            
    if printMatrix:
        # kontingensi tabel untuk kelas 0
        """
        plt.figure(figsize=(4,4))
        sns.heatmap([[tn0,fp0],[fn0,tp0]], annot=True, fmt='d', xticklabels=['false', 'true'],
                    yticklabels=['false', 'true'], cmap=plt.cm.Blues, cbar=False)
        plt.title("Class 0's Confusion Matrix")
        plt.xlabel('Predicted Label')
        plt.ylabel('True Label')
        plt.show()
        """
        
        # kontingensi tabel untuk kelas 1
        plt.figure(figsize=(6,6))
        global hitmap
        hitmap = sns.heatmap([[tn1,fp1],[fn1,tp1]], annot=True, fmt='d', xticklabels=['negatif', 'positif'],
                    yticklabels=['negatif', 'positif'], cmap=plt.cm.Blues, cbar=False)
        plt.title("Confusion Matrix\n0 = negatif, 1 = positif")
        plt.xlabel('Data Prediksi')
        plt.ylabel('Data Aktual')
        #plt.labels = ['d','whjat','ksaoks']
        #plt.show()
        #global showplt
        #showplt = plt.show()
        #showplt
        
        global trueneg1
        trueneg1=tn1
        global falsepos1
        falsepos1=fp1
        global falseneg1
        falseneg1=fn1
        global truepos1
        truepos1=tp1
        """
        figure
        figure = plt.figure(figsize=(6,6))
        ax = figure.subplots()
        conmat = pd.crosstab([[tn1,fp1],[fn1,tp1]], rownames = ['actual'], colnames = ['predicted'])
        sns.heatmap(conmat, square=True, cbar=False, ax=ax)
        """
    return (tp0, fp0, tn0, fn0), (tp1, fp1, tn1, fn1)
"""
def init_gui(root, update_function: Callable) ->FigureCanvasTkAgg:
    def event_key_press(event):
        print("you pressed {}".format(event.key))
        update_function()
        key_press_handler(event, canvas)

    # create empty figure and draw
    init_figure = create_figure()
    canvas = FigureCanvasTkAgg(init_figure, master=root)
    canvas.draw()
    canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)
    # call key press event
    canvas.mpl_connect("key_press_event", event_key_press)
    return canvas

def redraw_figure():
    figure = create_figure()
    canvas.figure = figure
    canvas.draw
    
sns.set()
canvas = init_gui(root, update_function=redraw_figure)
"""

def calc_metric(confMatrix0, confMatrix1):
    
    import pandas as pd
    tp0, fp0, tn0, fn0 = confMatrix0
    tp1, fp1, tn1, fn1 = confMatrix1
    
    # akurasi
    accuracy0 = (tp0+tn0)/(tp0+fp0+fn0+tn0)
    accuracy1 = (tp1+tn1)/(tp1+fp1+fn1+tn1)
    
    global accurate
    accurate = accuracy1

    # presisi
    precision0 = tp0/(tp0+fp1)
    precision1 = tp1/(tp1+fp0)
    
    # recall
    recall0 = tp0/(tp0+fn1)
    recall1 = tp1/(tp1+fn0)

    # f1-measure
    #f1_0 = (2*precision0*recall0)/(precision0+recall0)
    #f1_1 = (2*precision1*recall1)/(precision1+recall1)
    
    # micro-averaging
    tp_micro = tp0 + tp1
    fp_micro = fp0 + fp1
    tn_micro = tn0 + tn1
    fn_micro = fn0 + fn1
    
    #theaccurate
    global akurasi
    akurasi = (tp1+tn0)/(tp1+tn0+fp1+fn0)
    global presisi
    presisi = tp1/(fp1+tp1)*100
    global recall
    recall = tp1/(fn0+tp1)*100
    
    accuracy_micro = (tp_micro+tn_micro)/(tp_micro+fp_micro+fn_micro+tn_micro)
    precision_micro = (tp_micro)/(tp_micro+fp_micro)
    recall_micro = (tp_micro)/(tp_micro+fn_micro)
    f1_micro = (2 * precision_micro * recall_micro)/(precision_micro + recall_micro)
    
    # macro-averaging
    accuracy_macro = (accuracy0+accuracy1)/2
    precision_macro = (precision0+precision1)/2
    recall_macro = (recall0+recall1)/2
    #f1_macro = (f1_0+f1_1)/2
    global eval_df
    """
    eval_df = pd.DataFrame({'metriks evaluasi': ['akurasi','presisi','recall'], #'F1-measure'],
                            'hasil': [("%.0f%%" %(accuracy1*100)),
                                        ("%.0f%%" %(precision1*100)),
                                        ("%.0f%%" %(recall1*100))]})
                            #, precision1, recall1]}) #f1_1]})
                            #'micro': [accuracy_micro, precision_micro, recall_micro, f1_micro],
                            #'macro': [accuracy_macro, precision_macro, recall_macro, f1_macro]})
    #print(eval_df)
    #['akurasi (tingkat kebenaran antara nilai predilsi dengan nilai aktual)', 
    #'presisi (tingkat ketepatan informasi dari pengguna dengan jawaban sistem)', 
    #'recall (tingkat keberhasilan sistem dalam menemukan kembali sebuah sistem)'],
    """
    eval_df = pd.DataFrame({'metriks evaluasi': ['akurasi','presisi','recall'],
                            'positif': [("%.0f%%" %(accuracy1*100)),
                                        ("%.0f%%" %(precision1*100)),
                                        ("%.0f%%" %(recall1*100))],
                            'negatif': [("%.0f%%" %(accuracy0*100)), 
                                        ("%.0f%%" %(precision0*100)), 
                                        ("%.0f%%" %(recall0*100))],
                            'hasil rata': [("%.0f%%" %(accuracy_macro*100)), 
                                        ("%.0f%%" %(precision_macro*100)), 
                                        ("%.0f%%" %(recall_macro*100))]})
    

def evaluation(y_real, y_predict, printMatrix):
    
    # membuat 2 kontingensi tabel
    confMatrix0, confMatrix1 = conf_matrix(y_real, y_predict, printMatrix)
    
    # menghitung 4 metrik evaluasi untuk setiap kelas
    calc_metric(confMatrix0, confMatrix1)
#conf_matrix(y_real, y_predict, printMatrix)

def tren():
    filedatlatih = r"{}".format(datlatih['text'])
    #review = pd.read_excel(filedatlatih)
    filedatuji = r"{}".format(datuji['text'])
    #review = pd.read_excel(filedatlatih)
    #data_latih = option_1.get()
    #data_uji = option_2.get()
    #df, x_train,y_train,vocab,n_doc0,n_doc1,freq_0, freq_1,n_vocab0, n_vocab1, prob_c0, prob_c1, prob_w_c0, prob_w_c1  = traindata(data_latih)
    df, x_train,y_train,vocab,n_doc0,n_doc1,freq_0, freq_1,n_vocab0, n_vocab1, prob_c0, prob_c1, prob_w_c0, prob_w_c1  = traindata(filedatlatih)#(filedatlatih)
    """
    if filedatlatih=="":
        df, x_train,y_train,vocab,n_doc0,n_doc1,freq_0,freq_1,n_vocab0, n_vocab1, prob_c0, prob_c1, prob_w_c0, prob_w_c1  = traindata(conlat)
    else:
        df,x_train,y_train,vocab,n_doc0,n_doc1,freq_0,freq_1,n_vocab0, n_vocab1, prob_c0, prob_c1, prob_w_c0, prob_w_c1  = traindata(filedatlatih)
    """
    nVocab = 0
    global dokupos
    dokupos=n_doc1
    global dokuneg
    dokuneg=n_doc0
    global x_tren
    x_tren = x_train
    global katapos
    katapos=n_vocab1
    global kataneg
    kataneg=n_vocab0
    datapredict = training(x_train, y_train, nVocab, True)
    """
    if filedatuji=="":
        hasil_klasifikasi,y_test,x_test = testdata(conuj, datapredict)
    else:
        hasil_klasifikasi,y_test,x_test = testdata(filedatuji, datapredict)
    """
    hasil_klasifikasi,y_test,x_test = testdata(filedatuji, datapredict)
    #hasil_klasifikasi,y_test,x_test = testdata(data_uji, datapredict)#testdata("stopworduji.xlsx", datapredict)
    evaluation(y_test, hasil_klasifikasi, True)
    #print (x_train)
    #print("kosakata",vocab)
    #print("banyak dokumen",n_doc0,n_doc1)
    #print("banyak frekuensi",freq_0,freq_0)
    #print("banyak frekuensi",n_vocab0, n_vocab1)
    #print("probabilitas",prob_c0, prob_c1, prob_w_c0, prob_w_c1)
    probnegpos = pd.DataFrame({'Negatif':prob_w_c0 , 'Positif' : prob_w_c1})#({'Be':vocab})
    print (probnegpos)
    arrayprob = probnegpos.values
    neg_arrayp = arrayprob[:,0]
    pos_arrayp = arrayprob[:,1]
    
    global resfrektab
    resfrektab = pd.DataFrame({'Prob negatif':prob_w_c0 , 'Prob positif' : prob_w_c1})
                               #'Frek kata negatif':freq_0,'Frek kata positif':freq_1,
                               #({'Dokumen negatif': [n_doc0],
                               #'Dokumen positif': [n_doc1],
                               #'Frekuensi kata negatif':[freq_0],
                               #'Frekuensi kata positif':[freq_1]})
                        
    global vofrek
    vofrek = pd.DataFrame({"negatif":fre, "positif":fre1})
                        
    global dataktab
    probnegpose = pd.DataFrame({'Data_prediksi':y_test , 'Data_aktual' : hasil_klasifikasi})#({'Be':vocab})
    dataktual = probnegpose['Data_aktual'].map({1:'positif' , 0:'negatif'})
    datapredic = probnegpose['Data_prediksi'].map({1:'positif' , 0:'negatif'})
    dataktab = pd.DataFrame({'Data test':datest, 'Data aktual(Sebelum Diklasifikasi)':datapredic, 'Data prediksi NBC(Sesudah Diklasifikasi)':dataktual})
    dataktab
    
    global koskatabel
    koskatabel = pd.DataFrame({'Kosakata':vocab,'Probabilitas negatif':neg_arrayp, 'Probabilitas positif':pos_arrayp})
    

    confmat = Toplevel(root)
    teksi = "Confusion Matrix : "
    teksa = (fileuji)
    global teksb
    teksb = teksi+teksa
    confmat.title(teksb)
    confmat.geometry('350x350')
    confmat.state("normal")
    confmat.configure(bg="white")
    #label = Label(confmat, text = "chigga")
    #label.place(x=5, y=150)
    def init_gui(confmat, update_function: Callable) ->FigureCanvasTkAgg:
        def event_key_press(event):
            print("you pressed {}".format(event.key))
            update_function()
            key_press_handler(event, canvas)

        # create empty figure and draw
        init_figure = create_figure()
        canvas = FigureCanvasTkAgg(init_figure, master=confmat)
        canvas.draw()
        canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=6)
        # call key press event
        canvas.mpl_connect("key_press_event", event_key_press)
        return canvas

    def create_figure() ->Figure:
        figure = Figure(figsize=(6, 6))
        ax = figure.subplots()
        sns.heatmap([[trueneg1,falsepos1],[falseneg1,truepos1]], annot=True, fmt='d', xticklabels=['negatif', 'positif'],
                    yticklabels=['negatif', 'positif'], cmap=plt.cm.Blues, cbar=False,ax=ax)
        ptitle = plt.title("Confusion Matrix\n0 = negatif, 1 = positif")
        pxlabel = plt.xlabel('Data Prediksi')
        pylabel = plt.ylabel('Data Aktual')
        #plt.labels = ['d','whjat','ksaoks']
        #plt.show()
        hastes = "Hasil akurasi confusion matrix : "
        acr = accurate*100
        haskur = hastes+str("%.0f%%" %(acr))
        labelcm1 = Label(confmat,text="",background="white",height=2)
        labelcm1.pack()
        labelcm = Label(confmat,text=haskur, background="white")
        labelcm.place(x=10, y=8)
        
        Button(confmat, text = "Hasil lainnya", background = 'white',bd=0, fg='white',
               anchor = "ne",command=shwrincian).place(x=255, y=8)
        
        #sn.heatmap(confusion_matrix, square=True, cbar=False, ax=ax)
        return figure

    def redraw_figure():
        figure = create_figure()
        ptitle = create_figure()
        pxlabel = create_figure()
        pylabel = create_figure()
        canvas.figure = figure
        canvas.draw
    sns.set()
    canvas = init_gui(confmat, update_function=redraw_figure)
    menubar.entryconfig("Klasifikasi Manual", state="normal")
    nevbayes.entryconfig('Hasil Bag Of Words', state="normal")
    nevbayes.entryconfig('Hasil Frekuensi', state="normal")
    nevbayes.entryconfig('Hasil Probabilitas', state="normal")  
    nevbayes.entryconfig('Hasil Data Aktual Prediksi', state="normal")
    nevbayes.entryconfig('Hasil Rincian', state="normal")
    nevbayes.entryconfig('Bantuan', state="normal")
    
def shwrincian():
    resconmat = Toplevel(root)
    resconmat.title(teksb)
    resconmat.geometry('460x170')
    resconmat.configure(bg="aliceblue")
    resconmat.state("normal")
    f2resconmat = Frame(resconmat, height=400, width=400) 
    f2resconmat.pack(fill=BOTH,expand=1)
    tableresconmat = Table(f2resconmat, dataframe=eval_df, read_only=True)
    tableresconmat.columnconfigure(0,weight=3)
    tableresconmat.columnconfigure(1,weight=1)
    tableresconmat.show()

def bow():
    from sklearn.feature_extraction.text import CountVectorizer
    vec = CountVectorizer()
    count_kata = vec.fit_transform(x_tren)
    vec.fit(x_tren)
    vocabs = vec.get_feature_names ()
    X_train_transformed  = vec.transform(x_tren)
    X_train_transformed.toarray()
    bags = pd.DataFrame(X_train_transformed.toarray()  , columns = vec.get_feature_names())
    bow = Toplevel(root)
    bow .title("Hasil Bag of Words")
    bow .geometry('350x350')
    bow .configure(bg="aliceblue")
    bow .state("normal")
    bow2  = Frame(bow, height=400, width=400) 
    bow2.pack(fill=BOTH,expand=1)
    tablebow = Table(bow2 , dataframe=bags, read_only=True)
    tablebow.show()
   
def frekuensi():
    resfrek = Toplevel(root)
    resfrek .title("Hasil Frekuensi")
    resfrek .geometry('350x350')
    resfrek .configure(bg="aliceblue")
    resfrek .state("normal")
    resfrekf2  = Frame(resfrek, height=400, width=400) 
    resfrekf2.pack(fill=BOTH,expand=1)
    tableresfrek = Table(resfrekf2 , dataframe=koskatable, read_only=True)
    tableresfrek.show()
    
def kosakata():
    koskat = Toplevel(root)
    teks = "Hasil Probabilitas"
    koskat .title(teks)
    koskat .geometry('350x350')
    koskat .configure(bg="aliceblue")
    koskat .state("normal")
    koskatf2  = Frame(koskat, height=400, width=400) 
    koskatf2.pack(fill=BOTH,expand=1)
    tablekoskat = Table(koskatf2 ,dataframe = koskatabel, read_only=True) #dataframe=vofrek, read_only=True) #koskatabel, read_only=True)
    tablekoskat.show()
    
def dataktpred():
    datakt = Toplevel(root)
    tekd = "Hasil data aktual dan prediksi menggunakan data uji : "
    teks = (fileuji)
    teka = tekd+teks
    datakt.title(teka)
    datakt.geometry("350x350")
    datakt.configure(bg="aliceblue")
    datakt.state("normal")
    datakt2=Frame(datakt, height=400, width=400)
    datakt2.pack(fill=BOTH,expand=1)
    tabledatakt= Table(datakt2 , dataframe=dataktab, read_only=True)
    tabledatakt.show()
    
def rincian():
    df,x_train,y_train,vocab,n_doc0,n_doc1,freq_0,freq_1,n_vocab0, n_vocab1, prob_c0, prob_c1, prob_w_c0, prob_w_c1 = traindata(fileuji)
    rincian = Toplevel(root)
    rincian .title("Hasil rincian dari data")
    rincian .geometry('500x300')
    rincian .configure(bg=None)
    #rincian .state("normal")
    info = Label(rincian, text='Hasil rincian data')
    #info2 = Label(rincian, text=filenamed)
    info.pack()
    #info2.pack()
    
    global tabs
    tabs = ttk.Notebook(rincian)
    first = ttk.Frame(tabs)
    a = 5+1
    tabs.add(first, text='Jumlah dokumen')
    datname ="Data Latih : "
    comb = datname+filenamed
    combination = Label(first, text=comb)
    combination.place(x = 5, y= 5)
    
    dokpos = Label(first, text="Dokumen positif :")
    o = Label(first,text= dokupos)
    o.place(x = 115, y= 35)
    dokpos.place(x = 5, y= 35)
    
    dokneg = Label(first, text="Dokumen negatif :")#dokuneg)
    dokneg.place(x = 5, y= 55)
    i = Label(first,text= dokuneg)
    i.place(x = 115, y= 55)
    
    datuji = "Data Uji : "
    comb2 = datuji+fileuji
    combi2 = Label(first, text=comb2)
    combi2.place(x = 5, y= 85)
    
    dokujpos = Label(first, text="Dokumen positif :")
    o = Label(first,text= n_doc1)
    o.place(x = 115, y= 115)
    dokujpos.place(x = 5, y= 115)
    
    dokujneg = Label(first, text="Dokumen negatif :")#dokuneg)
    dokujneg.place(x = 5, y= 135)
    i = Label(first,text= n_doc0)
    i.place(x = 115, y= 135)
    
    second = ttk.Frame(tabs)
    tabs.add(second, text='Jumlah kemunculan kata')
    
    datname ="Data Latih : "
    comb = datname+filenamed
    combination = Label(second, text=comb)
    combination.place(x = 5, y= 5)
    katpos = Label(second, text="Kemunculan kata positif :")#, katapos)
    katpos.place(x = 5, y= 35)
    o2 = Label(second, text=katapos)
    o2.place(x = 150, y= 35)
    katpos = Label(second, text="Kemunculan kata negatif :")#, kataneg)
    katpos.place(x = 5, y= 55)
    i2 = Label(second, text=kataneg)
    i2.place(x = 150, y= 55)
    datuji = "Data Uji : "
    comb2 = datuji+fileuji
    combi2 = Label(second, text=comb2)
    combi2.place(x = 5, y= 85)
    dokujpos = Label(second, text="Kemunculan kata positif :")
    o = Label(second,text= n_vocab1)
    o.place(x = 150, y= 115)
    dokujpos.place(x = 5, y= 115)
    dokujneg = Label(second, text="Kemunculan kata negatif :")#dokuneg)
    dokujneg.place(x = 5, y= 135)
    i = Label(second,text= n_vocab0)
    i.place(x = 150, y= 135)
    
    global third
    """
    third = ttk.Frame(tabs)
    tabs.add(third, text="Klasifikasi manual")
    teks = Label(third, text="Input teks : ")
    teks.place(x = 5, y= 15)    
    enter = Entry(third, width = 65)
    enter.place(x = 70, y= 15)
    
    submit = Button(third, text="Proses", command = lambda : klasmanual2(enter))
    submit.place(x = 5, y= 40)
    
    hasyl = "Kalimat : "
    hasilprp = Label(third, text=hasyl)
    hasilprp.place(x = 5, y = 65)
    global haskmat
    haskmat = Label(third, text = "Tidak ada")
    haskmat.place(x=135, y=65)
    
    hasilprp = "Hasil preprocessing : "
    hasilcombprp = Label(third, text = hasilprp)
    hasilcombprp.place(x=5,y=85)
    global prephas
    prephas = Label(third, text = "Tidak ada")
    prephas.place(x=135, y=85)
    

    probres = Label(third, text = "Hasil klasifikasi : ")
    probres.place(x=5, y=105)
    global probhas
    probhas = Label(third, text = "Tidak ada")
    probhas.place(x=135, y=105)
    
    #submit2 = Button(third, text="Proses2", command = lambda : update)
    #submit2.place(x = 5, y= 140)
    #return tabs,third
    """
    fontstyle = ("arial",8,"normal")
    
    fourth = ttk.Frame(tabs)
    tabs.add(fourth, text="Klasifikasi manual teks")
    teks = Label(fourth , text="Input teks : ")
    teks.place(x = 5, y= 15)  
    global enter
    enter = Text(fourth , relief=GROOVE, width=55, height=5, wrap=WORD)
    enter.place(x = 135, y= 15)
    enter.configure(font = fontstyle)
    
    act = Button(fourth, text="Proses", command = lambda : replace(enter))
    act.place(x = 5, y= 70)
    
    outputeks = Label(fourth, text="Hasil preprocessing : ", anchor = 'w')
    outputeks.place(x = 5, y= 115)
    global outputw
    outputw = Text(fourth, relief=GROOVE, background='#f0f0f0', width=55, height=5, wrap=WORD)
    outputw.place(x = 135, y= 115)
    outputw.configure(font = fontstyle)
    
    nbteks = Label(fourth, text="Hasil klasifikasi : ")
    nbteks.place(x = 5, y= 200)
    global nbhasil
    nbhasil = Label(fourth, text ="-")
    nbhasil.place(x = 135, y= 200)
    
    """
    fifth = ttk.Frame(tabs)
    tabs.add(fifth, text="Probabilitas cheker")
    teks = Label(fifth, text="Input teks : ")
    teks.place(x = 5, y= 15)    
    enter = Entry(fifth, width = 65)
    enter.place(x = 70, y= 15)
    
    submit = Button(fifth, text="Cek", command = lambda : cheks(enter))
    submit.place(x = 5, y= 40)
    
    outpud = Label(fifth, text="Hasil teks : ")
    outpud.place(x = 5, y= 65)
    global outpud2
    outpud2 = Label(fifth, text="Tidak ada data")
    outpud2.place(x = 70, y = 65)
    
    third = ttk.Frame(tabs)
    tabs.add(third, text='Hasil probabilitas prior')
    katpos = Label(third, text="Kata yang mengandung positif :")#, katapos)
    katpos.place(x = 5, y= 15)
    o3 = Label(third, text=katapos)
    o3.place(x = 190, y= 15)
    katpos = Label(third, text="Kata yang mengandung negatif :")#, kataneg)
    katpos.place(x = 5, y= 35)
    i3 = Label(third, text=kataneg)
    i3.place(x = 190, y= 35)
    """
    tabs.pack(expand=1,fill="both")
    
    """
    liststyl = Frame(rincian, bd=1, relief = 'sunken', background="white")
    liststyl.pack(padx=10, pady=10, fill=None, expand=False)
    
    listinfo = Listbox(liststyl, width=20, height=10, borderwidhth=0,
                      highlightthickness=0, background=liststyle.cget("background"))
    vsb = Scrollbar(liststyl, orient="vertical",command=listinfo.yview)
    listinfo.configure(yscrollcommand=vsb)
    vsb.pack(side="right",fill="y")
    listinfo.pack(padx=10, pady=10, fill="both", expand=True)
    
    for i in range(20):
        listinfo.insert("end", "item #{}". format(i))
    """
    """
    info = Label(rincian, text='Hasil klasifikasi', anchor = 'e')
    info.pack()
    labrinci = LabelFrame(rincian, text='Hasil rincian dari data uji')#, padx = 150, pady = 20)#, anchor = 'e')
    info.config(font = ('arial',14))
    labrinci.pack(expand = 'yes', fill = 'both')
    doc = ttk.Label(labrinci, text='Hasil jumlah dokumen =', width = 60)#, anchor = 'e')
    doc.place(x = 5, y= 5)
    docpos = ttk.Label(labrinci, text='Dokumen positif =')#, anchor = 'e')
    docpos.place(x = 5, y= 15)#grid(row=7, column=0, padx=10, pady=15)#, sticky=N+W)
    
    labelteks = Label(rincian, text='Hasil klasifikasi', anchor = 'e')
    labelteks.config(font=('arial', 10))
    labelteks.pack()
    canvase = Canvas(rincian, highlightthickness=0, width = 320, height = 290,  relief = 'ridge', bg=None, bd=2)
    canvase.pack()
   
    ketrin = Label(rincian, text='Klasifikasi teks naivebayes', bg='#424242', fg='#9CB071')
    ketrin.config(font=('helvetica', 14))
    
    #canvase.create_window(225, 25, window=file_frame)
    dokumen = Label(rincian, text='Jumlah dokumen data') #anchor = 'e')
    dokumenpos = Label(rincian, text='Dokumen positif = 0as0das') #anchor = 'e')
    canvase.create_window(60, 40, window=dokumen)
    canvase.create_window(60, 80, window=dokumenpos)
    """
    #koskatf2  = Frame(koskat, height=400, width=400) 
    #koskatf2.pack(fill=BOTH,expand=1)
    #tablekoskat = Table(koskatf2 , dataframe=koskatabel, read_only=True)
    #tablekoskat.show()
    """
    onfmat = Toplevel(root)
    figure = Figure(figsize=(5, 4), dpi=100)
    plot = figure.add_subplot(1, 1, 1)
    canvas = FigureCanvasTkAgg(figure, onfmat)
    canvas.get_tk_widget().grid(row=0, column=0)
    #conftab.show()
    """
def manual(train):
    import pandas as pd
    import numpy as np
    from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
    from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
    import re 
    import string
    from nltk.corpus import stopwords
    
    x_trains = [train]
    #x_tests = [test]
    #x_trains
    
    # casefolding

    x_train_preprocess1 = []
    #x_test_preprocess1 = []

    for i in range(len(x_trains)):
        x_train_preprocess1.append(x_trains[i].lower())
    
    #for i in range(len(x_tests)):
        #x_test_preprocess1.append(x_tests[i].lower())
    
    # cleaning (hapus angka)
    
    x_train_preprocess2 = []
    #x_test_preprocess2 = []

    for i in range(len(x_train_preprocess1)):
        x_train_preprocess2.append(re.sub(r"\d+", "", x_train_preprocess1[i]))   
    
    #for i in range(len(x_test_preprocess1)):
        #x_test_preprocess2.append(re.sub(r"\d+", "", x_test_preprocess1[i]))
        
    # cleaning (hapus simbol)
        
    x_train_preprocess3 = []
    #x_test_preprocess3 = []

    for i in range(len(x_train_preprocess2)):
        x_train_preprocess3.append(x_train_preprocess2[i].translate(str.maketrans("","",string.punctuation)))
    
    #for i in range(len(x_train_preprocess2)):
        #x_test_preprocess3.append(x_test_preprocess2[i].translate(str.maketrans("","",string.punctuation)))
        
    # stopword
    stop_factory = StopWordRemoverFactory()
    more_stopword = ['aku','saya','dan','banyak']
    data = stop_factory.get_stop_words()+more_stopword
    
    sw_remover = StopWordRemoverFactory().create_stop_word_remover()
    x_train_preprocess4 = []
    #x_test_preprocess4 = []

    for i in range(len(x_train_preprocess3)):
        x_train_preprocess4.append(sw_remover.remove(x_train_preprocess3[i]))
    
    #for i in range(len(x_test_preprocess3)):
        #x_test_preprocess4.append(sw_remover.remove(x_test_preprocess3[i]))
        
    # normalisasi
    
    normalizad_word = pd.read_excel("normalisasi.xlsx")

    normalizad_word_dict = {}

    for index, row in normalizad_word.iterrows():
        if row[0] not in normalizad_word_dict:
            normalizad_word_dict[row[0]] = row[1] 

    def normalized_term(document):
        return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]
    x_train_preprocess5 = normalized_term(x_train_preprocess4)
    #x_test_preprocess5 = normalized_term(x_test_preprocess4)
    
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
 
    x_train_preprocess6 = []
    #x_test_preprocess6 = []

    for i in range(len(x_train_preprocess5)):
        x_train_preprocess6.append(stemmer.stem(x_train_preprocess5[i]))
    
    #for i in range(len(x_test_preprocess5)):
        #x_test_preprocess6.append(stemmer.stem(x_test_preprocess5[i]))
        
    return x_train_preprocess6
                             
def preprocessing(d):
    import re
    from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    d = d.lower()     
    d = re.sub(r'[^A-Za-z]', ' ', d)    
    d = d.split()    
    stoplist = (open('indo.txt', 'r').read().replace('\n', ' ')).split(" ")
    d = [i for i in d if i not in stoplist]    
    d = [i for i in d if len(i)>1]    
    d = [stemmer.stem(i) for i in d]
    str1 = " "
    d = (str1.join(d))
    return [d]
    
def testing2(testteks, model):
    
    from nltk.tokenize import word_tokenize
    
    y = []

    for t in testteks:
        tokens = word_tokenize(t)
        # probabilitas prior
        p0 = model.prob_c0
        ask1 = model.prob_c1
        p1 = model.prob_c1

        # menghitung probabilitas title t masuk ke kelas 0 dan kelas 1
        for w in tokens:
            if w in model.prob_w_c0:
                p0 = p0 * model.prob_w_c0[w]
            if w in model.prob_w_c1:
                p1 = p1 * model.prob_w_c1[w]
        print (p0,p1)
        # penentuan most likely class
        if p0 > p1:
            y.append("negatif")
        else:
            y.append("positif")

    return max(p0,p1),y

def cheks(key):
    yoy = key.get()
    teks = r"{}".format(yoy)
    outpud2.config(text = teks)

def klasmanual(entry1):
    filedatlatih = r"{}".format(datlatih['text'])
    #review = pd.read_excel(filedatlatih)
    #filedatuji = r"{}".format(datuji['text'])
    #review = pd.read_excel(filedatlatih)
    df,x_train,y_train,vocab,n_doc0,n_doc1,freq_0,freq_1,n_vocab0, n_vocab1, prob_c0, prob_c1, prob_w_c0, prob_w_c1  = traindata(filedatlatih)
    nVocab = 0
    yoy = entry1.get()
    teks = r"{}".format(yoy)
    #eje = {("hasil dari data",teks,"memiliki nilai sebesar = ")}
    eksts = preprocessing(teks)
    model_preprocess6 = training(x_train, y_train, nVocab, True)
    y_predict_preprocess6 = testing2(eksts, model_preprocess6)
    labelhasil = Label(inputeks, text="hasil dari kalimat", bg='#424242', fg='aliceblue')
    labelhasil.config(font=('helvetica', 14))
    labelhasil1 = Label(inputeks, text=teks, bg='#424242', fg='aliceblue', width = 450, height = 2)
    labelhasil1.config(font=('helvetica', 14))
    labelhasil2 = Label(inputeks, text="memiliki nilai sebesar" ,bg='#424242', fg='aliceblue')
    labelhasil2.config(font=('helvetica', 14))
    labelhasil3 = Label(inputeks, text=y_predict_preprocess6 ,bg='#424242', fg='aliceblue',width = 450)
    labelhasil3.config(font=('helvetica', 14))
    canvas1.create_window(350, 198, window=labelhasil)
    canvas1.create_window(350, 240, window=labelhasil1)
    canvas1.create_window(350, 280, window=labelhasil2)
    canvas1.create_window(350, 320, window=labelhasil3)
    print ("hasil dari data",teks,"memiliki nilai sebesar = ",y_predict_preprocess6)#,y_predict_preprocess6)

def replace(w):
    filedatlatih = r"{}".format(datlatih['text'])
    df,x_train,y_train,vocab,n_doc0,n_doc1,freq_0,freq_1,n_vocab0, n_vocab1, prob_c0, prob_c1, prob_w_c0, prob_w_c1  = traindata(filedatlatih)
    nVocab = 0
    yoy = w.get("1.0", END)
    teks = r"{}".format(yoy)
    eksts = preprocessing(teks)
    theprep = ''.join(map(str,eksts))
    preps = theprep.replace("!@#$","")
    model_preprocess6 = training(x_train, y_train, nVocab, True)
    y_predict_preprocess6 = testing2(eksts, model_preprocess6)
    nbhasil.config(text = y_predict_preprocess6)
    outputw.delete("1.0", END)
    outputw.insert(END, preps)
    
    if preps == "":
        nbhasil.config(text = "")
        
def replace2(k):
    yoy = k.get("1.0", END)
    teks = r"{}".format(yoy)
    #eje = {("hasil dari data",teks,"memiliki nilai sebesar = ")}
    eksts = preprocessing(teks)
    theprep = ''.join(map(str,eksts))
    preps = theprep.replace("!@#$","")
    outpute.delete("1.0", END)
    outpute.insert(END, preps)
    
def klasmanual2(entry2):
    filedatlatih = r"{}".format(datlatih['text'])
    #review = pd.read_excel(filedatlatih)
    #filedatuji = r"{}".format(datuji['text'])
    #review = pd.read_excel(filedatlatih)
    df,x_train,y_train,vocab,n_doc0,n_doc1,freq_0,freq_1,n_vocab0, n_vocab1, prob_c0, prob_c1, prob_w_c0, prob_w_c1  = traindata(filedatlatih)
    nVocab = 0
    yoy = entry2.get()
    teks = r"{}".format(yoy)
    #eje = {("hasil dari data",teks,"memiliki nilai sebesar = ")}
    eksts = preprocessing(teks)
    theprep = ''.join(map(str,eksts))
    preps = theprep.replace("!@#$","")
    model_preprocess6 = training(x_train, y_train, nVocab, True)
    y_predict_preprocess6 = testing2(eksts, model_preprocess6)
                    
    haskmat.config(text = teks)
    prephas.config(text = theprep)
    probhas.config(text = y_predict_preprocess6)
    
    if theprep == "":
        probhas.config(text = "Kosong")
        
    """
    hasyl = "Kalimat : "
    hasyyl = hasyl+teks
    hasilprp = Label(third, text=hasyl)
    hasilprp.place(x = 5, y = 65)
    haskmat = Label(third, text = teks)
    haskmat.place(x=135, y=65)
    
    hasilprp = "Hasil preprocessing : "
    hasilprp2 = ") adalah : "
    hasilprp3 = hasilprp+teks+hasilprp2
    hasilcombprp = Label(third, text = hasilprp)
    hasilcombprp.place(x=5,y=85)

    prephas = Label(third, text = eksts)
    prephas.place(x=135, y=85)
    probres = Label(third, text = "Hasil klasifikasi : ")
    probres.place(x=5, y=105)
    probhas = Label(third, text = y_predict_preprocess6)
    probhas.place(x=135, y=105)
    """
                    
    
    print ("hasil dari data",teks,"memiliki nilai sebesar = ",y_predict_preprocess6)#,y_predict_preprocess6)

def inputteks():
    global inputeks
    inputeks = Toplevel(root)
    inputeks.title("Klasifikasi Manual")
    #inputeks.geometry('350x350')
    inputeks.configure(bg="#424242")
    inputeks.state("normal")
    """
    menubar = Menu(inputeks)
    inputeks.config(menu=menubar)
    bantuan = Menu(inputeks, tearoff = 0)
    menubar.add_cascade(label ='Bantuan', menu = bantuan, command = bantu) #state=tk.DISABLED) #state=tk.DISABLED)
    bantuan.add_command(label = 'Info', command = inputteks)
    
    
    # specify size of window.
    inputeks.geometry("250x170")
  
    # Create text widget and specify size.
    T = Text(inputeks, height = 5, width = 52)
  
    # Create label
    l = Label(inputeks, text = "Fact of the Day")
    l.config(font =("Courier", 14))
  
    Fact = A man can be arrested in
    Italy for wearing a skirt in public
  
    # Create button for next text.
    b1 = Button(inputeks, text = "Next", )
  
    # Create an Exit button.
    b2 = Button(inputeks, text = "Exit",
                command = inputeks.destroy) 
  
    l.pack()
    T.pack()
    b1.pack()
    b2.pack()
  
    # Insert The Fact.
    T.insert(tk.END, Fact)
    """
  
    global canvas1
    canvas1 = Canvas(inputeks, highlightthickness=0, width = 700, height = 400,  relief = 'ridge', bg="#424242", bd=0)
    canvas1.pack()

    label1 = Label(inputeks, text='Klasifikasi teks naivebayes', bg='#424242', fg='aliceblue')
    label1.config(font=('helvetica', 14))
    canvas1.create_window(350, 25, window=label1)

    label2 = Label(inputeks, text='Input kalimat:', bg='#424242', fg='aliceblue')
    canvas1.create_window(350, 70, window=label2)
    #global entry
    entry1 = Entry(inputeks, width = 52, bd = 0, bg='darkgrey') #height = 5) 
    canvas1.create_window(350, 110, window=entry1)
    
    proses = Button(inputeks, text='Proses', bg='#424242', fg='aliceblue', command = lambda : klasmanual(entry1))
    canvas1.create_window(350, 160, window=proses)
    
    label2['font']=myFont
    proses['font']=myFont
        

    
def bantu():
    msg.showinfo('Info',
                'Klasifikasi untuk teks\nPengujian dilakukan dengan menggunakan data train serta menginputkan teks secara manual')
def bantuconfmat():
    msg.showinfo('Info',
                'akurasi (tingkat kebenaran antara nilai predilsi dengan nilai aktual)\npresisi (tingkat ketepatan informasi dari pengguna dengan jawaban sistem)\nrecall (tingkat keberhasilan sistem dalam menemukan kembali sebuah sistem)')
"""
def getSquareRoot ():
    
    x1 = entry1.get()
    
    label3 = Label(inputeks, text= 'The Square Root of ' + x1 + ' is:',font=('helvetica', 10))
    canvas1.create_window(200, 210, window=label3)
    
    label4 = Label(inputeks, text= float(x1)**0.5,font=('helvetica', 10, 'bold'))
    canvas1.create_window(200, 230, window=label4)
    
    button1 = Button(text='Get the Square Root', command=getSquareRoot, bg='brown', fg='white', font=('helvetica', 9, 'bold'))
    canvas1.create_window(200, 160, window=button1)
"""
def newtabel():
    canvastab = Canvas(root, highlightthickness=0, width = 400, height = 300,  relief = 'ridge', bg="lightblue", bd=0)
    canvastab.pack()
    click = Button(root, bg='aliceblue', anchor = "e", width = 60, height = 21, bd=0, command = lambda : canvastab.pack_forget()) #width = 200, height = 200)
    clicke = Button(root, bg='navyblue', anchor = "e", width = 30, height = 21, bd=0, command = lambda : canvastab.pack_forget())
    click.pack()
    clicke.pack()
    canvastab.create_window(20, 50, window=click)
    #canvasyntkts.create_window(200, 150, window=clicke)

#menu bar
menubar = Menu(root)
root.config(menu=menubar)
file = Menu(menubar, tearoff = 0)
menubar.add_cascade(label ='File', menu = file)
#file.add_command(label ='New File', command = None)
file.add_command(label ='Open File Data Preprocessing', command = File_dialog)
file.add_command(label ='Open File Data Latih', command = inputlatih)
file.add_command(label ='Open File Data Uji', command = inputuji)
file.add_command(label ='Bantuan', command = helpp)
#file.add_command(label ='Save', command = None)
file.add_separator()
file.add_command(label ='Exit', command = root.destroy)
        
edit = Menu(root, tearoff = 0)
menubar.add_cascade(label ='Preprocessing', menu = edit, state=tk.DISABLED)
edit.add_command(label ='Proses', command = prins)
edit.add_command(label ='Show Data', command = showtabel1)
edit.add_command(label ='Hasil Preprocessing', command = showtabel2)
edit.add_separator()
sub_menu = Menu(edit, tearoff=0)
sub_menu.add_command(label = 'Proses Data Uji', command = converd)
sub_menu.add_command(label = 'Show Data Uji', command = showcondat)
sub_menu.add_command(label = 'Proses Data Latih', command = converduji)
sub_menu.add_command(label = 'Show Data Latih', command = showconuji)
#edit.add_cascade(label ='Apply', menu=sub_menu)
edit.add_command(label ='Save Data', command = exportExcel)
edit.entryconfig(2, state=DISABLED)
edit.entryconfig(4, state=DISABLED)
edit.entryconfig(5, state=DISABLED)

nevbayes = Menu(root, tearoff=0)
menubar.add_cascade(label ='Klasifikasi Naive Bayes', menu = nevbayes, state=tk.DISABLED) #state=tk.DISABLED)
"""
sub_nevbayes = Menu(nevbayes, tearoff = 0)
option_1 = tk.StringVar()
option_2 = tk.StringVar()
sub_nevbayes.add_radiobutton(label="No Data", variable=option_1, value=datlatih)
sub_nevbayes.add_radiobutton(label="No Data", variable=option_1)#, value=conlat)
sub_nevbayes.add_separator()
sub_nevbayes.add_radiobutton(label="No Data",variable=option_2, value=datuji)
sub_nevbayes.add_radiobutton(label="No Data",variable=option_2)#, value=conuji)
sub_nevbayes.add_separator()
sub_nevbayes.add_command(label="Proses", command=tren)
nevbayes.add_cascade(label = 'Proses', menu=sub_nevbayes)
"""
nevbayes.add_cascade(label = 'Proses', command=tren)
nevbayes.add_command(label = 'Show Data Latih',command = showdatlatih)
nevbayes.add_command(label = 'Show Data Uji' ,command = showdatuji)
nevbayes.add_separator()
nevbayes.add_command(label ='Hasil Bag Of Words', command = bow)
nevbayes.add_command(label ='Hasil Frekuensi', command = frekuensi)
nevbayes.add_command(label ='Hasil Probabilitas', command = kosakata)  
nevbayes.add_command(label ='Hasil Data Aktual Prediksi', command = dataktpred)
nevbayes.add_command(label ='Hasil Rincian', command = rincian) 
nevbayes.add_separator()
nevbayes.add_command(label = 'Bantuan', command=bantuconfmat)
nevbayes.entryconfig(4, state=DISABLED)
nevbayes.entryconfig(5, state=DISABLED)
nevbayes.entryconfig(6, state=DISABLED)
nevbayes.entryconfig(7, state=DISABLED)
nevbayes.entryconfig(8, state=DISABLED)
nevbayes.entryconfig(10, state=DISABLED)


manualnb = Menu(root, tearoff = 0)
menubar.add_cascade(label ='Klasifikasi Manual', menu = manualnb, state=tk.DISABLED) #state=tk.DISABLED)
manualnb.add_command(label = 'Input Teks', command = inputteks)
manualnb.add_command(label = 'Bantuan', command = bantu)
#edit.add_command(label ='Paste', command = None)
#edit.add_command(label ='Select All', command = None)
#edit.add_separator()
#edit.add_command(label ='Find...', command = None)
#edit.add_command(label ='Find again', command = None)
#newWindow.config(menu = menubar)

#configure window
root.wm_title("Naive Bayes Python")
root.geometry('670x370')
root.configure(bg="aliceblue")
root.state("normal")
Showtemplabel = Entry(root);

"""btn = Button(root, text = 'Open File', bd = '0', command = File_dialog,
             bg = 'lightblue', fg = 'white')#, width=6, height=2)
btn2 = Button(root, text = 'Klasifikasi', bd = '0', 
             bg = 'lightblue', fg = 'white')#, width=6, height=2)"""

#define font
myFont = font.Font(family='Helvetica', size=15, weight='bold')
#yntktsfont = font.Font(family='Consolas', size=15, weight='bold')
fontinputan = font.Font(family='Helvetica', size=16, weight='bold')
"""
"""
#configure label
inputan = Label(root, text="APLIKASI KLASIFIKASI TEKS\nMENGGUNAKAN NAIVE BAYES", bd = '0',
                bg = 'aliceblue', fg = 'lightblue', anchor = "e")#, command = lambda : canvasyntkts.pack()) #command = lambda : canvasyntkts.pack_forget())
inputan.pack(side=TOP, pady = 8, padx = 2)#place(relx = 0.5, rely = 0.06, anchor = 'center')

#global cnvasyntkts
#canvasyntkts = Canvas(root, highlightthickness=0, width = 400, height = 300,  relief = 'ridge', bg="lightblue", bd=0)
#canvasyntkts.pack()
#yntkts = Button(root, text="/", 
               #bd = '0',fg = 'aliceblue', bg = 'aliceblue', ancho r = "e", command = lambda : canvasyntkts.pack_forget())#command = root.destroy)
#canvasyntkts.create_window(200, 150, window=yntkts)


#button configure
#btn.place(x=5, y=5)
#btn2.place(x=90, y=5)
#btn3.place(x=221, y=5)

inputan['font']=fontinputan
#yntkts['font']=yntktsfont
#file_frame['font']=myFont
label_files['font']=myFont

#btn['font']=myFont
#btn2['font']=myFont
#btn3['font']=myFont

root = mainloop()
